# Attribution Demo 

<a target="_blank" href="https://colab.research.google.com/github/safety-research/circuit-tracer/blob/main/demos/attribute_demo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In this demo, you'll learn how to load models and perform attribution on them.

In [8]:
#@title Colab Setup Environment

try:
    import google.colab
    !mkdir -p repository && cd repository && \
     git clone https://github.com/safety-research/circuit-tracer && \
     curl -LsSf https://astral.sh/uv/install.sh | sh && \
     uv pip install -e circuit-tracer/

    import sys
    from huggingface_hub import notebook_login
    sys.path.append('repository/circuit-tracer')
    sys.path.append('repository/circuit-tracer/demos')
    notebook_login(new_session=False)
    IN_COLAB = True
except ImportError:
    IN_COLAB = False
    # import sys
    # sys.path.insert(0, "/home/tu/circuit-tracer/circuit_tracer")

In [ ]:
from pathlib import Path
import torch

from circuit_tracer import ReplacementModel, attribute
from circuit_tracer.utils.create_graph_files import create_graph_files  
from circuit_tracer.graph import Graph, prune_graph, compute_graph_scores

First, load your model and transcoders by name. `model_name` is a normal HuggingFace / [TransformerLens](https://github.com/TransformerLensOrg/TransformerLens) model name; we'll use `google/gemma-2-2b`. We set `transcoder_name` to `gemma`, which is shorthand for the [Gemma Scope](https://arxiv.org/abs/2408.05147) transcoders; we take the transcoders with lowest L0 (mean # of active features) for each layer.

We additionally support `model_name = "meta-llama/Llama-3.2-1B"`, with `"llama"` transcoders; these are ReLU skip-transcoders that we trained, available [here](https://huggingface.co/mntss/skip-transcoder-Llama-3.2-1B-131k-nobos/tree/new-training).

If you want to use other models, you'll have to provide your own transcoders. To do this, set `transcoder_name` to point to your own configuration file, specifying the list of transcoders that you want to use. You can see `circuit_tracer/configs` for example configs.

In [ ]:
from huggingface_hub import login
import os

login(token= str(os.getenv("HUGGINGFACE_API_KEY")))

In [3]:
model_name = 'google/gemma-2-2b'
transcoder_name = "gemma" #"gemma" mntss/clt-gemma-2-2b-426k
model = ReplacementModel.from_pretrained(model_name, transcoder_name, dtype=torch.bfloat16)

Fetching 26 files:   0%|          | 0/26 [00:00<?, ?it/s]

KeyboardInterrupt: 

Next, set your attribution arguments.

In [9]:
prompt = "Fact: The capital of state containing Dallas is"  # What you want to get the graph for
max_n_logits = 10   # How many logits to attribute from, max. We attribute to min(max_n_logits, n_logits_to_reach_desired_log_prob); see below for the latter
desired_logit_prob = 0.95  # Attribution will attribute from the minimum number of logits needed to reach this probability mass (or max_n_logits, whichever is lower)
max_feature_nodes = 8192  # Only attribute from this number of feature nodes, max. Lower is faster, but you will lose more of the graph. None means no limit.
batch_size=256  # Batch size when attributing
offload='disk' if IN_COLAB else 'cpu' # Offload various parts of the model during attribution to save memory. Can be 'disk', 'cpu', or None (keep on GPU)
verbose = True  # Whether to display a tqdm progress bar and timing report

Then, just run attribution!

In [ ]:
graph = attribute(
    prompt=prompt,
    model=model,
    max_n_logits=max_n_logits,
    desired_logit_prob=desired_logit_prob,
    batch_size=batch_size,
    max_feature_nodes=max_feature_nodes,
    offload=offload,
    verbose=verbose
)

Phase 0: Precomputing activations and vectors
Precomputation completed in 0.51s
Found 8296 active features
Phase 1: Running forward pass
Forward pass completed in 0.09s
Phase 2: Building input vectors
Selected 10 logits with cumulative probability 0.7148
Will include 8192 of 8296 feature nodes
Input vectors built in 1.81s
Phase 3: Computing logit attributions
<sys>:0: UserWarning: Full backward hook is firing when gradients are computed with respect to module outputs since no inputs require gradients. See https://docs.pytorch.org/docs/main/generated/torch.nn.Module.html#torch.nn.Module.register_full_backward_hook for more details.
Logit attributions completed in 0.08s
Phase 4: Computing feature attributions
Feature influence computation: 100%|██████████| 8192/8192 [00:01<00:00, 4368.04it/s]
Feature attributions completed in 1.88s
Attribution completed in 9.61s


We now have a graph object! We can save it as a .pt file, but be warned that it's large (~167MB).

In [5]:
graph_dir = 'graphs'
graph_name = 'austin.pt'
graph_dir = Path(graph_dir)
graph_dir.mkdir(exist_ok=True)
graph_path = graph_dir / graph_name

# graph.to_pt(graph_path)

Given this object, we can create the graph files that we need to visualize the graph. Give it a slug (name), and set the node / edge thresholds for pruning. Pruning removes unimportant nodes and edges from your graph; lower thresholds (i.e., more aggressive pruning) results in smaller graphs. These may be easier to interpret, but explain less of the model's behavior.

In [ ]:
slug = "dallas-austin"  # this is the name that you assign to the graph
graph_file_dir = './graph_files'  # where to write the graph files. no need to make this one; create_graph_files does that for you
node_threshold=0.8  # keep only the minimum # of nodes whose cumulative influence is >= 0.8
edge_threshold=0.98  # keep only the minimum # of edges whose cumulative influence is >= 0.98

create_graph_files(
    graph_or_path=graph_path,  # the graph to create files for
    slug=slug,
    output_path=graph_file_dir,
    node_threshold=node_threshold,
    edge_threshold=edge_threshold
)

NameError: name 'graph_path' is not defined

Now, you can visualize the graph using the following commands! This will spin up a local server to act as the frontend.

**If you're running this notebook on a remote server, make sure that you set up port forwarding, so that the chosen port is accessible on your local machine too.**

You can select nodes by clicking on them. Ctrl/Cmd+Click on nodes to pin and unpin them to your subgraph. G+Click on nodes in the subgraph to group them together into a supernode; G+Click on the X next to a supernode to dissolve it. Click on the edit button to edit node descriptions, and click on supernode description to edit that.

In [4]:
from circuit_tracer.frontend.local_server import serve

port = 8046
server = serve(data_dir='./graph_files/', port=port)

if IN_COLAB:
    from google.colab import output as colab_output  # noqa
    colab_output.serve_kernel_port_as_iframe(port, path='/index.html', height='800px', cache_in_notebook=True)
else:
    from IPython.display import IFrame
    print(f"Use the IFrame below, or open your graph here: f'http://localhost:{port}/index.html'")
    # display(IFrame(src=f'http://localhost:{port}/index.html', width='100%', height='800px'))


OSError: [Errno 98] Address already in use

In [11]:
display(IFrame(src=f'http://localhost:{port}/index.html', width='100%', height='800px'))

Once you're done, you can stop the server with the following command.

In [11]:
server.stop()

NameError: name 'server' is not defined

Congrats, you're done! Go to `intervention_demo.ipynb` to see how to perform interventions, or check out `gemma_demo.ipynb` and `llama_demo.ipynb` for examples of worked-out test examples. Read on for a bit more info aabout the Graph class and pruning.

## Graphs

Earlier, you created a graph object. Its adjacency matrix / edge weights are stored in `graph.adjacency_matrix` in a dense format; rows are target nodes and columns are source nodes. The first `len(graph.real_features)` entries of the matrix represent features; the `i`th entry corresponds to the `i`th feature in `graph.real_features`, given in `(layer, position, feature_idx)` format. The next `graph.cfg.n_layers * graph.n_pos` entries are error_nodes. The next `graph.n_pos` entries are token nodes. The final `len(graph.logit_tokens)` entries are logit nodes.

The value of the cell `graph.adjacency_matrix[target, source]` is the direct effect of the source node on the target node. That is, it tells you how much the target node's value would change if the source node were set to 0, while holding the attention patterns, layernorm denominators, and other feature activations constatnt. Thus, if the target node is a feature, this tells you how much the target feature would change; if the target node is a logit, this tells you how much the (de-meaned) value of the logit would change.

Note that `gemma-2-2b` is model (family) that uses logit softcapping. This means that a softcap function, `softcap(x) = t * tanh(x/t)` is used to constrain the logits to fall within (-t, t); `gemma-2-2b` uses `t=30`. For such models, we predict the change in logits *pre-softcap*, as the nonlinearity introduced by softcapping would cause our attribution to yield incorrect / approximate direct effect values.

### Pruning
Given a graph, you might want to prune it, as it will otherwise contain many low-impact nodes and edges that clutter the circuit diagram while adding little information. We enable you to prune nodes by absolute influence, i.e. the total impact that the nodes have on the logits, direct and indirect. The default threshold is 0.8: this means we will keep the minimum number of nodes required to capture 80% of all logit effects. Similarly, the edge_threshold, by default 0.98, means that we will keep the minimum number of edges required to capture 98% of all logit effects.

# Visualization

In [10]:
import numpy as np
from circuit_tracer.subgraph.pruning import trim_graph
from circuit_tracer.subgraph.grouping import greedy_grouping
from circuit_tracer.subgraph.distance import build_distance_graph_from_clerp

graph_file_path = "graph_files/gemma-clt-fact-dallas-austin_2025-09-25T14-52-21-776Z.json"

In [11]:
G, attr = trim_graph(graph_file_path, crit="topk", top_k = 5)
print(f"Created graph with {G.number_of_nodes()} nodes and {G.number_of_edges()} edges.")
print("Nodes:", list(G.nodes(data=False))[:5])    
# distance_graph = np.random.rand(G.number_of_nodes(), G.number_of_nodes())
distance_graph = build_distance_graph_from_clerp(G, attr, progress=True, normalize=True)
groups, merged_G = greedy_grouping(G, distance_graph, attr, num_groups=20)
print(f"Formed {len(groups)} clusters.")
for i, c in enumerate(groups):
    print(f"Cluster {i}: {c}")

Created graph with 26 nodes and 73 edges.
Nodes: ['27_22605_10', 'E_26865_9', '20_44686_10', '1_52044_10', 'E_603_10']


Embedding nodes: 100%|██████████| 1/1 [00:00<00:00, 52.99it/s]

Formed 20 clusters.
Cluster 0: ('27_22605_10',)
Cluster 1: ('E_26865_9',)
Cluster 2: ('20_44686_10',)
Cluster 3: ('E_603_10',)
Cluster 4: ('E_2_0',)
Cluster 5: ('E_235292_2',)
Cluster 6: ('E_6037_4',)
Cluster 7: ('E_2329_7',)
Cluster 8: ('E_714_3',)
Cluster 9: ('E_10751_8',)
Cluster 10: ('0_95057_10',)
Cluster 11: ('E_18143_1',)
Cluster 12: ('E_573_6',)
Cluster 13: ('1_57794_9',)
Cluster 14: ('1_12928_10', '1_12928_9')
Cluster 15: ('1_52044_10', '1_52044_9')
Cluster 16: ('1_72774_9', '1_72774_10')
Cluster 17: ('0_91045_10', '0_32742_9')
Cluster 18: ('16_89970_9', '7_24743_9')
Cluster 19: ('0_40780_10', '0_95475_9')


In [12]:
from circuit_tracer.subgraph.visualization import visualize_clusters

visualize_clusters(
    G,
    draw=True,
    filename='subgraphs/subgraph.png',
    label_fn=lambda node: attr[node].get('clerp') if attr[node].get('clerp') != "" else str(node)
)
visualize_clusters(
    merged_G,
    draw=True,
    filename='subgraphs/merged_subgraph.png',
    label_fn=lambda tuple_node: " + ".join(attr[node].get('clerp') if attr[node].get('clerp') != "" else str(node) for node in tuple_node)
)

Layer 0: Emb: "  Dallas", Emb: "  is", Emb: " <bos>", Emb: " :", Emb: "  capital", Emb: "  state", Emb: "  The", Emb: "  containing", Emb: " Fact", Emb: "  the"
Layer 1: code snippets, punctuation, is, places, health/cancer, code related, Code/licensing snippets
Layer 2: code related,  AssemblyCulture, ble,  AssemblyCulture, Code/licensing snippets
Layer 3: Texas, Texas legal matters
Layer 4: Texas locations/legal contexts
Layer 5: Output " Austin" (p=0.439)
Saved visualization to subgraphs/subgraph.png
Layer 0: Emb: "  Dallas", Emb: "  is", Emb: " <bos>", Emb: " :", Emb: "  capital", Emb: "  state", Emb: "  The", Emb: "  containing", Emb: " Fact", Emb: "  the"
Layer 1: punctuation, is + places, code snippets + health/cancer
Layer 2: ble,  AssemblyCulture +  AssemblyCulture, code related + code related, Code/licensing snippets + Code/licensing snippets
Layer 3: Texas + Texas legal matters
Layer 4: Texas locations/legal contexts
Layer 5: Output " Austin" (p=0.439)
Saved visualization to

[[('E_26865_9',),
  ('E_603_10',),
  ('E_2_0',),
  ('E_235292_2',),
  ('E_6037_4',),
  ('E_2329_7',),
  ('E_714_3',),
  ('E_10751_8',),
  ('E_18143_1',),
  ('E_573_6',)],
 [('0_95057_10',), ('0_91045_10', '0_32742_9'), ('0_40780_10', '0_95475_9')],
 [('1_57794_9',),
  ('1_12928_10', '1_12928_9'),
  ('1_52044_10', '1_52044_9'),
  ('1_72774_9', '1_72774_10')],
 [('16_89970_9', '7_24743_9')],
 [('20_44686_10',)],
 [('27_22605_10',)]]

In [ ]:
from graph_visualization import visualize_intervention_graph

visualize_intervention_graph(merged_G, prompt = prompt)